## Análise de Sentimentos

### Tweets

In [1]:
#importações

import pandas as pd

In [2]:
#Abertura da base de dados

df = pd.read_csv('data/Tweets_Mg.csv', sep=',')
df

,Unnamed: 0,Created At,Text,Geo Coordinates.latitude,Geo Coordinates.longitude,User Location,Username,User Screen Name,Retweet Count,Classificacao,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,0,Sun Jan 08 01:22:05 +0000 2017,���⛪ @ Catedral de Santo Antônio - Governador ...,NaN,NaN,Brasil,Leonardo C Schneider,LeoCSchneider,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Sun Jan 08 01:49:01 +0000 2017,"� @ Governador Valadares, Minas Gerais https:/...",-41.9333,-18.85,NaN,Wândell,klefnews,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Sun Jan 08 01:01:46 +0000 2017,"�� @ Governador Valadares, Minas Gerais https:...",-41.9333,-18.85,NaN,Wândell,klefnews,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Wed Jan 04 21:43:51 +0000 2017,��� https://t.co/BnDsO34qK0,NaN,NaN,NaN,Ana estudando,estudandoconcur,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Mon Jan 09 15:08:21 +0000 2017,��� PSOL vai questionar aumento de vereadores ...,NaN,NaN,NaN,Emily,Milly777,0,Negativo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8194,8194,Thu Feb 09 11:48:07 +0000 2017,"Trio é preso suspeito de roubo, tráfico e abus...",NaN,NaN,NaN,Ana Lúcia,lapiseirapentel,0,Positivo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8195,8195,Thu Feb 09 12:10:19 +0000 2017,"Trio é preso suspeito de roubo, tráfico e abus...",NaN,NaN,Belo Horizonte - Minas Gerais,Marcelo Rezende,Televans,0,Positivo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8196,8196,Thu Feb 09 12:04:17 +0000 2017,"Trio é preso suspeito de roubo, tráfico e abus...",NaN,NaN,Guarulhos - SP,Leonardo Nascimento,leonardogru,0,Positivo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8197,8197,Thu Feb 09 12:10:04 +0000 2017,"Trio é preso suspeito de roubo, tráfico e abus...",NaN,NaN,Brasil Natal/RN,Lucas Medeiros �©™,parabolicalucas,0,Positivo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Pré-Processamento dos Dados

#### 1) Checar se as classes estão balanceadas
#### 2) Checar se há textos duplicados. Como se trata de Tweets, é bem possível
#### 3) Remoção de Stopwords do idioma relacionado aos textos
#### 4) Reduzir o vocabulário levando as palavras para seu radical
####        4.1) Stemming ou Lemming

In [3]:
#Checando o balanceamento das classes
#Estão mais ou menos parecidas

df['Classificacao'].value_counts()

Positivo    3300
Neutro      2453
Negativo    2446
Name: Classificacao, dtype: int64

In [4]:
#Como se trata de Tweets, há muitos casos em que o tweet é repetido, apenas mudando mínimas coisas ou até não mudando nada.
#Como é o caso do retweet.
#Dessa forma, é preciso realizar a checagem de registros duplicados

df.drop_duplicates(['Text'], inplace=True) #É possível passar uma coluna específica para realizar a checagem de duplicados nos parâmetros



#Antes 8199
#Depois 5765

In [5]:
df.Classificacao.count()

5765

In [6]:
#Como se trata de um problema de análise de sentimentos, os únicos atributos que vão ser necessários, é o que contém o texto
#e o atributo que contém a rotulação. Todos os outros não agregam valor nesse contexto.

#Separação do que importa

textos = df['Text']

sentimento = df['Classificacao']
sentimento

0         Neutro
1         Neutro
2         Neutro
3         Neutro
4       Negativo
          ...   
8194    Positivo
8195    Positivo
8196    Positivo
8197    Positivo
8198    Positivo
Name: Classificacao, Length: 5765, dtype: object

In [7]:
#importação do ToolKit de PNL. NLTK
#Download dos pacotes que vamos utilizar

import nltk

nltk.download('stopwords')
nltk.download('rslp')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\preda\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\preda\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\preda\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\preda\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Funções de Pré Processamento de Dados

### 1) Remover Stopwords.
##### - Carrega Cria a lista das Stopwords da lingua portuguesa
##### - Faz a separação das palavras da string e itera sobre ela, checando se cada palavra está dentro da lista de stopwords
##### - Retorna a junção de todas as palavras em uma string só com o join()

In [66]:
stopwords = set(nltk.corpus.stopwords.words('portuguese'))
stopwords

{'a',
 'ao',
 'aos',
 'aquela',
 'aquelas',
 'aquele',
 'aqueles',
 'aquilo',
 'as',
 'até',
 'com',
 'como',
 'da',
 'das',
 'de',
 'dela',
 'delas',
 'dele',
 'deles',
 'depois',
 'do',
 'dos',
 'e',
 'ela',
 'elas',
 'ele',
 'eles',
 'em',
 'entre',
 'era',
 'eram',
 'essa',
 'essas',
 'esse',
 'esses',
 'esta',
 'estamos',
 'estas',
 'estava',
 'estavam',
 'este',
 'esteja',
 'estejam',
 'estejamos',
 'estes',
 'esteve',
 'estive',
 'estivemos',
 'estiver',
 'estivera',
 'estiveram',
 'estiverem',
 'estivermos',
 'estivesse',
 'estivessem',
 'estivéramos',
 'estivéssemos',
 'estou',
 'está',
 'estávamos',
 'estão',
 'eu',
 'foi',
 'fomos',
 'for',
 'fora',
 'foram',
 'forem',
 'formos',
 'fosse',
 'fossem',
 'fui',
 'fôramos',
 'fôssemos',
 'haja',
 'hajam',
 'hajamos',
 'havemos',
 'hei',
 'houve',
 'houvemos',
 'houver',
 'houvera',
 'houveram',
 'houverei',
 'houverem',
 'houveremos',
 'houveria',
 'houveriam',
 'houvermos',
 'houverá',
 'houverão',
 'houveríamos',
 'houvesse',


In [71]:
def removeStopwords(texto):
    
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    vocabulario = [i for i in texto.split() if not i in stopwords]
    
    return (" ").join(vocabulario)

In [83]:
string = 'oi tudo bem como voce vai a noite?'
mystring = 'https://oi.com.br'

string1 = removeStopwords(string) #Removeu o 'a' pois considerou como uma stopwords da lingua portuguesa
string1

'oi tudo bem voce vai noite?'

## Função de Stemming

#### Reduz a palavra ao seu radical (prefixo) com o objetivo de diminuir o vocabulário de um texto, pois podem levar palavras diferentes ao mesmo prefixo. Porém, no Stemming, a redução ocorre e o resultado dela, não necessariamente representa uma palavra existente na língua.

#### 1) Instancia o Objeto do Stemmer do nltk
#### 2) Itera sobre as palavras de uma string separadas pelo split()
#### 3) Aplica a função stem() do objeto criado, fazendo o append na lista de vocabulario
#### 4) Para não retornar uma lista, faz-se o join() e junta tudo em uma única string

In [74]:

def Stemming(texto):
    
    stemmer = nltk.stem.RSLPStemmer()
    vocabulario = []
    for i in texto.split():
        vocabulario.append(stemmer.stem(i))
    
    return (" ".join(vocabulario))

In [77]:
string2 = Stemming(string) #Removeu muitas coisas deixando so o radical
string2

'oi tud bem com voc vai a noite?'

## Função para Limpeza dos Textos

#### Remove pontuações, links, coisas que não agregam valor na tarefa de análise de sentimentos.
#### Será feiro a partir de expressões regulares a substituição desses itens irregulares por espaço

In [90]:
import re

def limpar_textos(texto):
    
    vocabulario = re.sub(r"http\S", "", texto).lower().replace('.', ' ').replace('-', ' ').replace(';', ' ').replace(':', ' ').replace('/','')
    return (vocabulario)
    

In [92]:
string3 = limpar_textos(mystring)
string3

' oi com br'

## Função Lemmatization

#### Também atua como um redutor de palavras. Porém, diferente do Stemmer o Lemming reduz as palavras para um radical que reflete a realidade, ou seja, todas as palavras que o lemming reduz existem de forma concreta no dicionário. Dessa forma, possuem o mesmo objetivo, de reduzir o vocabulário.
#### O passo a passo é semelhante

####

In [98]:
#Depende muito do idioma, não tem lemmatizer no wordnet para portugues

from nltk.stem import WordNetLemmatizer

def Lemmatization(texto):
    lemmatizer = WordNetLemmatizer()
    
    vocabulario = []
    
    for i in texto.split():
        vocabulario.append(lemmatizer.lemmatize(i))
        
        return (" ".join(vocabulario))
        
    

In [101]:
string4 = Lemmatization("Os carros são bonitos")
string4

'Os'

## Função de PreProcessamento Completo 

#### Inclui todos os passos

In [105]:
def preProcessing(texto):
    #Cria objeto stemming
    stemmer = nltk.stem.RSLPStemmer()
    
    #Faz a limpeza com expressão regular substituindo
    textoLimpo = re.sub(r"http\S", "", texto).lower().replace('.', ' ').replace('-', ' ').replace(';', ' ').replace(':', ' ').replace('/','')
    
    #lista de stopwords
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    
    #aplicando o stemmer a cada elemento do texto
    vocabulario = [stemmer.stem(i) for i in textoLimpo.split() if not i in stopwords]
    
    return (" ".join(vocabulario))

In [21]:
#Textos pré- processados

textos = [preProcessing(i) for i in textos]
textos

NameError: name 'preProcessing' is not defined

## Tokenização

#### A maioria dos tokenizadores padrão apenas separam as palavras por espaço. Porém, cada caso pode ser diferente e no caso de Tweets, como é esse, há muito o uso de Hastags e emojis. Dessa forma, é melhor usar um tokenizador mais específico

In [8]:
from nltk.tokenize import word_tokenize

tweet = "Que dia lindo #poderosa, ;)" #identifica o # separado da palavra e os emojis também, como sendo tokens separados

word_tokenize(tweet)

['Que', 'dia', 'lindo', '#', 'poderosa', ',', ';', ')']

In [9]:
from nltk.tokenize import TweetTokenizer

#ANALISE DE TWITTER TEM QUE USAR ESSE TOKENIZADOR

#Como em análise de sentimento, essas coisas são importantes, é bom q sejam identificadas juntas para análista
tokenizer = TweetTokenizer()

tokenizer.tokenize(tweet)

['Que', 'dia', 'lindo', '#poderosa', ',', ';)']

## Criando o Modelo

In [10]:
#Vetorização do texto, estilo Bag of Words, criando o vocabulário e atribuindo um valor numerico para cada palavra que seria sua frequencia
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer="word", tokenizer = tokenizer.tokenize) #passa o tokenizador que já foi criado

freq_words = vectorizer.fit_transform(textos)
freq_words.shape #matriz esparsa 

(5765, 13361)

In [11]:
freq_words.A

array([[0, 0, 0, ..., 0, 0, 3],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 2],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

## Multinomial Naive Bayes

In [12]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB() #Algoritmo Multinomial Naive Bayes
model.fit(freq_words, sentimento)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

#### Para fazer os testes, precisa aplicar nos dados de testes o mesmo passo a passo usado para treinar o modelo
#### Nesse caso, só vetorizar mesmo

In [17]:
# Realizando testes com o modelo recém treinado

testes = ['Eu não gostei disso', 
          'Eu gosto disso', 
          'Estou muito triste para falar disso!!!', 
          'Eu amo você']

In [18]:
freq_testes = vectorizer.transform(testes) #Vectorizer ja está treinado com os tex
freq_testes.shape

(4, 13361)

In [19]:
result = model.predict(freq_testes)
result

array(['Neutro', 'Neutro', 'Neutro', 'Neutro'], dtype='<U8')

In [20]:
#Classificação

for t, c in zip (testes, model.predict(freq_testes)):
    print(t + ", " + c)

Eu não gostei disso, Neutro
Eu gosto disso, Neutro
Estou muito triste para falar disso!!!, Neutro
Eu amo você, Neutro


In [27]:
#Como está usando Naive Bayes, é possivel imporimir a probabilidade de ser de cada classe
#É atribuido a classe que possuir a maior probabilidade

print(model.classes_)
model.predict_proba(freq_testes).round(2)

['Negativo' 'Neutro' 'Positivo']


array([[0.1 , 0.9 , 0.01],
       [0.12, 0.86, 0.02],
       [0.01, 0.99, 0.  ],
       [0.02, 0.98, 0.01]])

## Tags de Negação

#### No modelo de Bag of Words, às vezes é complicado para fazer a diferenciação de duas frases iguais, porém uma contendo um "não", pois muda pouquíssima coisa.
#### O Bag of Words não consegue fazer bem a distinção e saber que um "não" pode estar negativando uma frase inteira.
#### Para minimizar isso, pode ser acrescentado uma tag de negação toda vez que o algoritmo encontrar um não na base de dados.
#### Quando é encontrada um "não" , é colocada em todas as palavras seguintes uma tag de negação e quando treinar o modelo, dará um peso maior negativo àquela frase. Aumenta o vocabulário mas deve melhorar a precisão.

#### Ajuda a classificar frases com inversão de sentimento com mais precisão
#### Exemplo:
#### - Eu gosto de comer carne
#### - Eu não gosto de comer carne

## Função para Adicionar as Tags

In [28]:
def tag_neg(texto):
    tags_neg = ['não', 'not']
    neg_detected = False
    vocabulario_neg = []
    vocabulario = texto.split()
    
    for i in vocabulario:
        i = i.lower()
        
        if neg_detected == True:
            i = i + '_NEG'
        
        if i in tags_neg:
            neg_detected = True
        
        vocabulario_neg.append(i)
    
    return (" ".join(vocabulario_neg))

In [30]:
#Teste com frase
#Adicionou as tags, aumentando o vocabulario, porém, dando mais peso negativo para esta frase

tag_neg("Eu não gosto de queijo")

'eu não gosto_NEG de_NEG queijo_NEG'

## Criar modelos com Pipeline


#### Pipeline é um objeto que encadeia tarefas.
#### É interessante para reduzir código e automatizar fluxos
#### Vantagem: pode-se criar varios pipelines mudando apenas alguns parâmetros e executar ao mesmo tempo para ver qual é o melhor

In [32]:
from sklearn.pipeline import Pipeline

## Naive Bayes

In [34]:
pipeline = Pipeline([
    ('count', CountVectorizer()),
    ('classifier', MultinomialNB())
])

In [36]:
pipeline_neg = Pipeline([
    ('count', CountVectorizer(tokenizer = lambda text: tag_neg(text))),
    ('classifier', MultinomialNB())
])

In [39]:
#treinamento

pipeline.fit(textos, sentimento) #pode-se ver os parãmetros para poder alterar

Pipeline(memory=None,
         steps=[('count',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('classifier',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [41]:
pipeline_neg.fit(textos,sentimento)

Pipeline(memory=None,
         steps=[('count',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function <lambda> at 0x000001C7F6827828>,
                                 vocabulary=None)),
                ('classifier',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [43]:
pipeline_neg.steps #etapas do pipeline

[('count',
  CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                  dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                  lowercase=True, max_df=1.0, max_features=None, min_df=1,
                  ngram_range=(1, 1), preprocessor=None, stop_words=None,
                  strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                  tokenizer=<function <lambda> at 0x000001C7F6827828>,
                  vocabulary=None)),
 ('classifier', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]

## SVM (Support Vector Machine)

In [64]:
from sklearn import svm

pipeline_svm = Pipeline([
    ('count', CountVectorizer()),
    ('classifier', svm.SVC(kernel='linear'))
])

In [65]:
pipeline_neg = Pipeline([
    ('count', CountVectorizer(tokenizer = lambda text: tag_neg(text))),
    ('classifier', svm.SVC(kernel='linear'))
])

## Validando os modelos com Validação Cruzada

In [56]:
from sklearn.model_selection import cross_val_predict
import sklearn.metrics as metrics

In [66]:
#Validação cruzada com cv = 10

resultado = cross_val_predict(pipeline, textos, sentimento, cv=10)

In [67]:
#Acurácia média

metrics.accuracy_score(sentimento, resultado)

0.8716392020815265

In [69]:
sentimentos = ['Positivo', 'Negativo', 'Neutro']

print(metrics.classification_report(sentimento, resultado, sentimentos))

              precision    recall  f1-score   support

    Positivo       0.95      0.89      0.92      2840
    Negativo       0.80      0.87      0.83       951
      Neutro       0.81      0.84      0.83      1974

    accuracy                           0.87      5765
   macro avg       0.85      0.87      0.86      5765
weighted avg       0.88      0.87      0.87      5765



In [70]:

resultado_neg = cross_val_predict(pipeline_neg,textos,sentimento,cv=10)

In [71]:
#Acurácia média

metrics.accuracy_score(sentimento, resultado_neg)

0.7694709453599307

## Avaliando modelo com Brigrama

#### bigramas são conjuntos de duas palavras. Cada token vai ser a junção de duas palavras
#### Quanto maior o valor de n em ngrama, mais contexto é passado para o algoritmo, porém não fica tão preciso

In [75]:
vectorize_bigram = CountVectorizer(ngram_range=(2,2)) #Passando a range para usar 2 palavras por token
freq_bigram = vectorize_bigram.fit_transform(textos)

modelo = MultinomialNB()
modelo.fit(freq_bigram,sentimento)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [79]:
result = cross_val_predict(modelo, freq_bigram, sentimento, cv=10)

In [81]:
metrics.accuracy_score(sentimento, result)

0.8412836079791848

# Considerações Finais

#### 1) Tentar outros features para agregar o modelo (foi usado apenas o texto puro com algumas transformações, não teve nenhum outro feature). Exemplo: tamanho do texto, emoticons.... etc
#### 2) Se tiver uma base APENAS com duas classes (positiva e negativa) o SVM Linear tende a funcionar muito bem